<a href="https://colab.research.google.com/github/Azzamjiul/machine-learning/blob/master/Stunting_in_Bangladesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preprocessing

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from pandas import read_stata
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils.multiclass import type_of_target
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
house_url = "gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/BDHR72FL.DTA"
children_url = "gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/BDKR72FL.DTA"
women_url = "gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/BDIR72FL.DTA"
house = read_stata(house_url)
children = read_stata(children_url)
women = read_stata(women_url)

In [31]:
house_2 = house.rename(columns={'hv001':'v001','hv002':'v002'})
children_house = pd.merge(children, house_2, how='inner')
print(children_house.shape)

(7886, 2778)


In [32]:
children_house_women = pd.merge(children_house, women, how='inner')
print(children_house_women.shape)

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:1108: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


(7886, 6787)


In [33]:
df = children_house_women[children_house_women['hw1'] < 60]
print(df.shape)

(7507, 6787)


In [0]:
boys_0_2 = pd.read_csv('gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/who/lhfa_boys_0_2_zscores.txt', sep="\t")
girls_0_2 = pd.read_csv('gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/who/lhfa_girls_0_2_zscores.txt', sep="\t")
boys_2_5 = pd.read_csv('gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/who/lhfa_boys_2_5_zscores.txt', sep="\t")
girls_2_5 = pd.read_csv('gdrive/My Drive/Data Azzam - Umum/Folder Azzam/Skripsi/Bangladesh/who/lhfa_girls_2_5_zscores.txt', sep="\t")
boys_2_5.columns = ['Month', 'L', 'M2', 'S', 'SD', 'SD3neg', 'SD2neg', 'SD1neg','SD0', 'SD1', 'SD2', 'SD3']
girls_2_5.columns = ['Month', 'L', 'M2', 'S', 'SD', 'SD3neg', 'SD2neg', 'SD1neg','SD0', 'SD1', 'SD2', 'SD3']

In [0]:
boys = pd.concat([boys_0_2,boys_2_5], sort=True, join='outer')
girls = pd.concat([girls_0_2,girls_2_5], sort=True, join='outer')

In [0]:
def hitung_HAZ (umur, tinggi, sex):
    median = 0
    SD = 0
    who = 0
    if sex  == 'female':
        who = girls[girls['Month']==umur]
    else:
        who = boys[boys['Month']==umur]
        
    if umur > 24:
        median = who.iloc[0]['M2']
        SD = who.iloc[0]['SD']
    else:
        median = who.iloc[0]['M']
        SD = who.iloc[0]['SD']

    HAZ_manual = (tinggi- median)/ SD
    
    if HAZ_manual < -2:
        return 1
#         return HAZ_manual
    else:
        return 0
#         return HAZ_manual

In [0]:
i = 0
stunting = []

for index,b in df.iterrows():
    i = i + 1
    sex = b['b4']
    age = b['hw1']

    s = str(b['hw3'])
    
    if s.replace('.','',1).isdigit():
        height = b['hw3']/10
        HAZ = hitung_HAZ(age,height,sex)
    else:
        HAZ = -99
    
    stunting.append(HAZ)

In [38]:
stunting_se = pd.Series(stunting)
df['stunting_status'] = stunting_se.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
df = df[df['stunting_status'] != -99]
print(df.shape)

(7134, 6788)


In [0]:
columns = ['hc61_1','hv270','hv014','m15','hw1','hc64_1','hv220','h9','hv024','v717','b4','stunting_status']

In [41]:
data = df[columns]
data

,hc61_1,hv270,hv014,m15,hw1,hc64_1,hv220,h9,hv024,v717,b4,stunting_status
0,primary,poorest,1,respondent's home,13.0,2.0,37,vaccination date on card,barisal,not working,female,0
3,primary,poorest,1,NaN,47.0,2.0,70,vaccination date on card,barisal,not working,female,0
4,primary,poorer,1,respondent's home,23.0,3.0,34,vaccination date on card,barisal,agricultural - self employed,female,0
5,primary,poorer,1,respondent's home,11.0,1.0,22,vaccination date on card,barisal,skilled manual,female,0
6,primary,middle,1,NaN,51.0,4.0,51,vaccination date on card,barisal,agricultural - employee,female,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7881,primary,middle,3,respondent's home,22.0,2.0,90,vaccination date on card,sylhet,not working,male,1
7882,secondary,richest,1,NaN,49.0,1.0,28,vaccination date on card,sylhet,not working,male,0
7883,primary,richer,1,NaN,53.0,3.0,42,vaccination date on card,sylhet,not working,male,1
7884,primary,middle,0,respondent's home,32.0,1.0,36,reported by mother,sylhet,not working,male,1


In [42]:
num_missing = (np.isnan(data[['hv014','hw1','hc64_1']])).sum()
print(num_missing)

hv014      0
hw1        0
hc64_1    67
dtype: int64


In [43]:
data['hc64_1'].fillna(data['hc64_1'].mean(), inplace=True)
num_missing = (np.isnan(data[['hv014','hw1','hc64_1']])).sum()
print(num_missing)

hv014     0
hw1       0
hc64_1    0
dtype: int64


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [44]:
data.isnull().sum()

hc61_1               67
hv270                 0
hv014                 0
m15                2818
hw1                   0
hc64_1                0
hv220                 0
h9                   15
hv024                 0
v717                 10
b4                    0
stunting_status       0
dtype: int64

In [0]:
my_imputer = SimpleImputer(strategy='most_frequent')
imputed_data = pd.DataFrame(my_imputer.fit_transform(data))
imputed_data.columns = data.columns

In [46]:
imputed_data.isnull().sum()

hc61_1             0
hv270              0
hv014              0
m15                0
hw1                0
hc64_1             0
hv220              0
h9                 0
hv024              0
v717               0
b4                 0
stunting_status    0
dtype: int64

In [47]:
# ['hv014','hw1','hc64_1']
hehe = pd.get_dummies(imputed_data[['hc61_1', 'hv270', 'm15', 'hv220', 'h9', 'hv024', 'v717', 'b4']])
hehe.shape

(7134, 117)

In [0]:
X = imputed_data[['hv014','hw1','hc64_1']]

In [0]:
X = pd.concat([X, hehe], axis=1)

In [0]:
Y = imputed_data['stunting_status']

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

(6420, 120) (6420,)
(714, 120) (714,)


In [0]:
def create_baseline():
  model = Sequential()
  model.add(Dense(120, input_dim=120, activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  print(model.summary())
  return model

In [53]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print(results)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 120)               14520     
_________________________________________________________________
dense_62 (Dense)             (None, 60)                7260      
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 61        
Total params: 21,841
Trainable params: 21,841
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
5778/5778 [==============================] - 2s 281us/step - loss: 0.6479 - accuracy: 0.6352
Epoch 2/100
5778/5778 [==============================] - 1s 257us/step - loss: 0.6175 - accuracy: 0.6653
Epoch 3/100
5778/5778 [==============================] - 1s 254us/step - loss: 0.6054 - accuracy: 0.6672
Epoch 4/100
5778/5778 [==============================] - 2s

In [54]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('ann', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X_test, y_test, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print(results)

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 120)               14520     
_________________________________________________________________
dense_92 (Dense)             (None, 60)                7260      
_________________________________________________________________
dense_93 (Dense)             (None, 1)                 61        
Total params: 21,841
Trainable params: 21,841
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
642/642 [==============================] - 0s 361us/step - loss: 0.6685 - accuracy: 0.6028
Epoch 2/100
642/642 [==============================] - 0s 239us/step - loss: 0.5720 - accuracy: 0.6963
Epoch 3/100
642/642 [==============================] - 0s 256us/step - loss: 0.5122 - accuracy: 0.7477
Epoch 4/100
642/642 [==============================] - 0s 240us/s